In [1]:
# # Install necessary packages
# !pip install streamlit pandas pickle5 psycopg2-binary numpy

# # Import statements
# import streamlit as st
# import pandas as pd
# from datetime import datetime
# import pickle

# import time
# import json
# # import pyodbc  # Uncomment if you decide to use pyodbc later
# import psycopg2
# import pandas as pd
# from psycopg2 import sql
# import traceback
# import numpy as np
# from psycopg2.extras import execute_values

# # Example usage of a few imports

# # Create a simple pandas DataFrame
# df = pd.DataFrame({
#     'A': [1, 2, 3],
#     'B': [4, 5, 6]
# })

# # Display DataFrame in Streamlit app
# st.write("Sample DataFrame:")
# st.dataframe(df)

# # Connect to a PostgreSQL database (replace with your own connection details)
# try:
#     conn = psycopg2.connect(
#         dbname="your_dbname",
#         user="your_username",
#         password="your_password",
#         host="your_host",
#         port="your_port"
#     )
#     st.write("Database connection successful")
# except Exception as e:
#     st.write("Database connection failed")
#     st.write(e)
#     st.write(traceback.format_exc())
# finally:
#     if 'conn' in locals():
#         conn.close()



In [2]:
# !pip install pandas psycopg2-binary numpy pickle5




In [7]:
# import pandas as pd
from datetime import datetime
import pickle
import time
# import psycopg2
# from psycopg2 import sql
import traceback
import numpy as np
# from psycopg2.extras import execute_values


In [8]:
def fix_omc_id(data):
    if data.count("-"):
        try:
            return str(int(data.split('-')[1]))
        except Exception as e:
            return data.split('-')[1]
    return data
def fix_vin(data):
    return str(int(data.replace('omc',"")))
def fule_level_consumption(fuel_level_df):
    fuelleval_dict={}
    for vin in fuel_level_df['vin'].unique():
        tmp_df=fuel_level_df[fuel_level_df['vin']==vin].sort_values(by='event_time_stamp')
        d_fuel_level=tmp_df['fuelleval'].astype(float)
        fuelleval_dict[vin] = d_fuel_level.to_list()[0]-d_fuel_level.to_list()[-1]
        print("*",end="")
    # Convert dictionary to DataFrame
    df = pd.DataFrame.from_dict(fuelleval_dict, orient='index', columns=['fuel_level'])
    
    # Reset the index to make the dictionary keys as a column
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'vin'}, inplace=True)
    return df
            

In [9]:
import pickle
import pandas as pd

# Define a function to load pickle files with exception handling
def load_pickle(file_path):
    try:
        with open(file_path, 'rb') as f:
            return pickle.load(f)
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

# File paths
file_paths = {
    'dbug_df': 'dbug_df.plk',
    'df_raw': 'df_raw.plk',
    'dgrh_df': 'dgrh_df.plk',
    'fuel_level_df': 'fuel_level_df.plk',
    'load_survey_df': 'load_survey_df.plk',
    'master_df': 'master_df.plk',
    'tower_load_df': 'tower_load_df.plk',
    'omc_hour_df': 'omc_hour_df.plk'
}

# Load DataFrames
dataframes = {}
for key, path in file_paths.items():
    dataframes[key] = load_pickle(path)
    print(f"{key} loaded successfully.")

# Accessing DataFrames
dbug_df = dataframes['dbug_df']
df_raw = dataframes['df_raw']
dgrh_df = dataframes['dgrh_df']
fuel_level_df = dataframes['fuel_level_df']
# fuel_level_df.loc[fuel_level_df['vin']=='omc890','fuelleval']=13
load_survey_df = dataframes['load_survey_df']
master_df = dataframes['master_df']
tower_load_df = dataframes['tower_load_df']
omc_hour_df = dataframes['omc_hour_df']

# Verify loading
for key, df in dataframes.items():
    if df is not None:
        print(f"{key} DataFrame:")
        print(df.head())
    else:
        print(f"{key} DataFrame not loaded.")


Error loading dbug_df.plk: [Errno 2] No such file or directory: 'dbug_df.plk'
dbug_df loaded successfully.
Error loading df_raw.plk: [Errno 2] No such file or directory: 'df_raw.plk'
df_raw loaded successfully.
Error loading dgrh_df.plk: [Errno 2] No such file or directory: 'dgrh_df.plk'
dgrh_df loaded successfully.
Error loading fuel_level_df.plk: [Errno 2] No such file or directory: 'fuel_level_df.plk'
fuel_level_df loaded successfully.
Error loading load_survey_df.plk: [Errno 2] No such file or directory: 'load_survey_df.plk'
load_survey_df loaded successfully.
Error loading master_df.plk: [Errno 2] No such file or directory: 'master_df.plk'
master_df loaded successfully.
Error loading tower_load_df.plk: [Errno 2] No such file or directory: 'tower_load_df.plk'
tower_load_df loaded successfully.
Error loading omc_hour_df.plk: [Errno 2] No such file or directory: 'omc_hour_df.plk'
omc_hour_df loaded successfully.
dbug_df DataFrame not loaded.
df_raw DataFrame not loaded.
dgrh_df DataF

In [6]:
master_df['vin']=master_df['vin'].astype(str).apply(lambda x: 'omc' + x.split('.')[0])
master_df['plant_id']=master_df['plant_id'].astype(str).apply(lambda x: x.split('.')[0])
tower_load_df['omc_id'] =tower_load_df['omc_id'].astype(str).apply(fix_omc_id)
master_df=master_df[['vin','plant_id','site_name','total_cap_in_kwp','cap_in_kva','cap_in_kw','fuel_cap_in_litres','total_ah','battery_cap_in_kwh']]
merged_df = pd.merge(df_raw, master_df, on='vin', how='left')
merged_df['solar_gen_factor'] = merged_df['solar_gen'] / merged_df['total_cap_in_kwp']
result = pd.merge(merged_df, tower_load_df.rename(columns={'omc_id': 'plant_id'}), on='plant_id', how='outer')
result['Batt Losses in KWH'] = result['battery_charging'] - result['battery_discharging'] 
result = pd.merge(result, dgrh_df, on='vin', how='outer')
result['DGRH'] = result['running_duration'].astype(float)  / 60
fl_df=fule_level_consumption(fuel_level_df)
fl_df.loc[fl_df['vin']=='omc890','fuelleval']=13
result = pd.merge(result, fl_df, on='vin', how='outer')
result['Diesel Cons in Litrs'] = (result['fuel_level'] /100 ) * result['fuel_cap_in_litres']
result['Trans Loss in %'] = ( (result['plant_load'] - result['energy_diff']) / result['plant_load'] ) * 100
result['Solar short fall in KWH'] = (result['plant_load'] - result['solar_gen']) + 0.2 * result['solar_gen'] + 0.2 * (result['plant_load'] - result['plant_load'])
result['CPH'] =  result['Diesel Cons in Litrs'] / result['DGRH']
result['DG Gen / Hr in KWH'] = result['dg_kwh']/result['DGRH']
result['Exp DGRH'] = result['Solar short fall in KWH'] / result['DG Gen / Hr in KWH']
result['Exp DG Gen in KW'] = result['Exp DGRH'] * result['DG Gen / Hr in KWH']
result['Ext DGRH'] = result['DGRH']  - result['Exp DGRH']
result['Extra fuel cons in Ltrs'] = result['CPH'] * result['Ext DGRH']
omc_hour_df['plant_id'] =omc_hour_df['plant_id'].astype(str).apply(fix_omc_id)
result = pd.merge(result, omc_hour_df, on='plant_id', how='outer')
cost_per_unit=23
cost_of_dpl=89
result.rename(columns={
    'plant_id':'Site ID',
    'total_cap_in_kwp':'Solar Cap in KWp',
    'total_cap_in_kwp':'Solar Cap in KWp',
    'cap_in_kva':'DG Cap in KVA',
    'solar_gen':'Solar Gen in KWH',
    'energy_diff':'Load in KWH @ Tower',
    'plant_load':'Load in KWH @ Plant',
    'battery_charging':'Battery Chg in KWH',
    'battery_discharging':'Batt Dischg in KWH',
    'dg_kwh':'DG Gen in KWH',
    'total_supply_hours':'No of Hours Load supplied by OMC',
    'site_name':'Plant Name',
    'solar_gen_factor':'Solar Gen Factor',
        },inplace=True)


result=result[['vin','Site ID',
'Plant Name',
'Solar Cap in KWp',
'DG Cap in KVA',
'Solar Gen in KWH',
'Load in KWH @ Tower',
'Load in KWH @ Plant',
'Battery Chg in KWH',
'Batt Dischg in KWH',
'Batt Losses in KWH',
'DGRH',
'DG Gen in KWH',
'Diesel Cons in Litrs',
'Solar Gen Factor',
'Trans Loss in %',
'Solar short fall in KWH',
'CPH',
'DG Gen / Hr in KWH',
'Exp DG Gen in KW',
'Exp DGRH',
'Ext DGRH',
'Extra fuel cons in Ltrs',
'No of Hours Load supplied by OMC',]]

clip_zero_list=['Batt Losses in KWH','Ext DGRH','Extra fuel cons in Ltrs','Solar short fall in KWH']
result[clip_zero_list] = result[clip_zero_list].clip(lower=0)
result['Revenue in INR'] = result['Load in KWH @ Tower'] * cost_per_unit
result.dropna(subset=['Plant Name','Site ID'], inplace=True)
result['Diesel Expenses in INR'] = result['Diesel Cons in Litrs'] * cost_of_dpl
result['Other Expenses in INR'] = 128
result['Plant expenses in INR'] = result['Other Expenses in INR'] + result['Diesel Expenses in INR']
result['Loss or Profit %'] = ((result['Revenue in INR'] - result['Plant expenses in INR'] ) / result['Revenue in INR']) * 100

TypeError: 'NoneType' object is not subscriptable

In [12]:

fuel_level_df.loc[fuel_level_df['vin']=='omc890','fuelleval']=13

In [8]:
fuel_level_df.isna().sum()

fuelleval           28090
vin                     0
event_time_stamp        0
dtype: int64

In [10]:
fuel_level_df[fuel_level_df['vin']=='omc890' ]

,fuelleval,vin,event_time_stamp
285,39,omc890,2024-07-04 00:01:26.790
528,39,omc890,2024-07-04 00:02:49.067
666,39,omc890,2024-07-04 00:03:27.127
1111,38,omc890,2024-07-04 00:11:28.440
1404,39,omc890,2024-07-04 00:05:27.503
...,...,...,...
255476,86,omc890,2024-07-04 23:52:26.117
256012,86,omc890,2024-07-04 23:54:26.393
256260,27,omc890,2024-07-04 18:33:36.277
256406,86,omc890,2024-07-04 23:56:26.693


In [12]:
fl_df = fule_level_consumption(fuel_level_df)

********************************************************************************************************************************************************************************************************************************************************************************************************

In [13]:
fl_df

,vin,fuel_level
0,omc854,8.0
1,omc895,-57.0
2,omc561,2.0
3,omc627,NaN
4,omc536,0.0
...,...,...
291,omc329,NaN
292,omc580,2.0
293,omc998,0.0
294,omc726,0.0


In [14]:
fl_df[fl_df['vin']=='omc890' ]

,vin,fuel_level
221,omc890,-47.0


In [5]:
result

,vin,Site ID,Plant Name,Solar Cap in KWp,DG Cap in KVA,Solar Gen in KWH,Load in KWH @ Tower,Load in KWH @ Plant,Battery Chg in KWH,Batt Dischg in KWH,...,Exp DG Gen in KW,Exp DGRH,Ext DGRH,Extra fuel cons in Ltrs,No of Hours Load supplied by OMC,Revenue in INR,Diesel Expenses in INR,Other Expenses in INR,Plant expenses in INR,Loss or Profit %
30,omc890,148,Ajrauli,33.800,25.0,78.8,150.04,159.8,94.7,72.8,...,96.76,11.043261,0.000000,36.974185,23.17,3450.92,-10457.5,128,-10329.5,399.325977
31,omc120,149,Judupur,25.165,25.0,69.4,103.56,114.2,42.4,42.3,...,58.68,4.300929,0.749071,4.820755,12.08,2381.88,2892.5,128,3020.5,-26.811594
32,omc171,150,Pakdipur,22.680,25.0,50.4,NaN,69.9,31.4,28.0,...,29.58,2.371981,1.028019,4.535377,NaN,NaN,1335.0,128,1463.0,NaN
33,omc228,151,Narsinghpur,27.300,25.0,40.9,38.66,40.0,39.3,22.4,...,7.28,0.577778,1.422222,7.111111,3.83,889.18,890.0,128,1018.0,-14.487505
34,omc917,152,Arthar,27.300,25.0,59.2,16.20,17.2,59.4,16.0,...,-30.16,-2.028700,3.528700,17.643498,2.5,372.60,667.5,128,795.5,-113.499732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,omc511,284,Saidapur,34.020,25.0,87.0,55.34,61.4,33.8,24.0,...,NaN,NaN,NaN,NaN,8.33,1272.82,0.0,128,128.0,89.943590
294,omc310,285,Ban Gawan,19.845,25.0,59.2,61.87,65.0,50.2,48.8,...,17.64,1.080000,0.120000,0.000000,10.5,1423.01,-21360.0,128,-21232.0,1592.048545
295,omc395,286,Kushaila,19.845,25.0,69.3,139.40,147.1,75.3,71.5,...,91.66,5.985595,0.714405,4.265107,23.42,3206.20,3560.0,128,3688.0,-15.027135
296,omc862,287,Ralpur,19.845,25.0,60.8,71.60,73.9,63.7,50.4,...,25.26,2.000496,1.349504,6.042553,10.42,1646.80,1335.0,128,1463.0,11.161040


In [6]:
# Display 'Site ID' and 'Diesel Cons in Litrs' columns
diesel_consumption_with_site_id = result[['Site ID','Plant Name','Diesel Cons in Litrs']]
print(diesel_consumption_with_site_id)


    Site ID   Plant Name  Diesel Cons in Litrs
30      148      Ajrauli                -117.5
31      149      Judupur                  32.5
32      150     Pakdipur                  15.0
33      151  Narsinghpur                  10.0
34      152      Arthar                    7.5
..      ...          ...                   ...
293     284     Saidapur                   0.0
294     285    Ban Gawan                -240.0
295     286     Kushaila                  40.0
296     287       Ralpur                  15.0
297     288     Sujaganj                -155.0

[245 rows x 3 columns]


In [9]:
# Display the columns of the result DataFrame
print(result.columns)


Index(['vin', 'Site ID', 'Plant Name', 'Solar Cap in KWp', 'DG Cap in KVA',
       'Solar Gen in KWH', 'Load in KWH @ Tower', 'Load in KWH @ Plant',
       'Battery Chg in KWH', 'Batt Dischg in KWH', 'Batt Losses in KWH',
       'DGRH', 'DG Gen in KWH', 'Diesel Cons in Litrs', 'Solar Gen Factor',
       'Trans Loss in %', 'Solar short fall in KWH', 'CPH',
       'DG Gen / Hr in KWH', 'Exp DG Gen in KW', 'Exp DGRH', 'Ext DGRH',
       'Extra fuel cons in Ltrs', 'No of Hours Load supplied by OMC',
       'Revenue in INR', 'Diesel Expenses in INR', 'Other Expenses in INR',
       'Plant expenses in INR', 'Loss or Profit %'],
      dtype='object')


In [10]:
# Display some of the 'Diesel Cons in Litrs' values
print(result[['Site ID', 'Diesel Cons in Litrs']].head())


   Site ID  Diesel Cons in Litrs
30     148                -117.5
31     149                  32.5
32     150                  15.0
33     151                  10.0
34     152                   7.5


In [11]:
fuel_level_df['fuelleval'] =  fuel_level_df['fuelleval'].astype(float)

In [10]:
datax=[]
for i in  fuel_level_df[fuel_level_df['vin'] == 'omc890']['fuelleval'].to_list():
    datax.append(i)

In [ ]:
df

In [11]:
datax

['39',
 '39',
 '39',
 '38',
 '39',
 '38',
 '39',
 '38',
 '38',
 '38',
 '38',
 '38',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '37',
 '36',
 '37',
 '36',
 '37',
 '36',
 '36',
 '36',
 '36',
 '36',
 '36',
 '36',
 '36',
 '35',
 '35',
 '35',
 '35',
 '35',
 '35',
 '35',
 '34',
 '35',
 '34',
 '34',
 '35',
 '34',
 '34',
 '34',
 '34',
 '34',
 '34',
 '34',
 '34',
 '34',
 '34',
 '34',
 '34',
 '34',
 '34',

In [33]:
from collections import Counter
value_counts = dict(Counter(datax))

In [35]:
value_counts


{39.0: 5,
 38.0: 7,
 37.0: 95,
 36.0: 10,
 35.0: 9,
 34.0: 75,
 33.0: 12,
 32.0: 108,
 31.0: 194,
 30.0: 60,
 29.0: 34,
 28.0: 11,
 27.0: 9,
 26.0: 27,
 41.0: 2,
 74.0: 2,
 57.0: 1,
 93.0: 2,
 60.0: 1,
 75.0: 1,
 91.0: 10,
 90.0: 50,
 89.0: 8,
 88.0: 7,
 87.0: 48,
 86.0: 10}

In [36]:
filtered_dict = {k: v for k, v in value_counts.items() if v > 2}

# Step 2: Find consecutive differences where the difference is 1 or 2
sorted_keys = sorted(filtered_dict.keys(), reverse=True)
consecutive_differences = []

for i in range(len(sorted_keys) - 1):
    if sorted_keys[i] - sorted_keys[i + 1] in {1, 2}:
        consecutive_differences.append(sorted_keys[i] - sorted_keys[i + 1])

# Step 3: Calculate the sum of these differences
sum_of_differences = sum(consecutive_differences)

print("Filtered Dictionary:", filtered_dict)
print("Sum of Consecutive Differences:", sum_of_differences)

Filtered Dictionary: {39.0: 5, 38.0: 7, 37.0: 95, 36.0: 10, 35.0: 9, 34.0: 75, 33.0: 12, 32.0: 108, 31.0: 194, 30.0: 60, 29.0: 34, 28.0: 11, 27.0: 9, 26.0: 27, 91.0: 10, 90.0: 50, 89.0: 8, 88.0: 7, 87.0: 48, 86.0: 10}
Sum of Consecutive Differences: 18.0


In [7]:
# for i in  fuel_level_df[fuel_level_df['vin'] == 'omc890']['fuel_cap_in_litres'].to_list():
#     print(i)
fl_df.loc[fl_df['vin']=='omc890','fuelleval']=18

In [8]:
fl_df.loc[fl_df['vin']=='omc890']

,vin,fuel_level,fuelleval
221,omc890,-47.0,18.0


In [13]:
result[result['Site ID']=='148']

,vin,Site ID,Plant Name,Solar Cap in KWp,DG Cap in KVA,Solar Gen in KWH,Load in KWH @ Tower,Load in KWH @ Plant,Battery Chg in KWH,Batt Dischg in KWH,...,Exp DG Gen in KW,Exp DGRH,Ext DGRH,Extra fuel cons in Ltrs,No of Hours Load supplied by OMC,Revenue in INR,Diesel Expenses in INR,Other Expenses in INR,Plant expenses in INR,Loss or Profit %
30,omc890,148,Ajrauli,33.8,25.0,78.8,150.04,159.8,94.7,72.8,...,96.76,11.043261,0.0,36.974185,23.17,3450.92,-10457.5,128,-10329.5,399.325977


In [26]:
fuel_level_df


,fuelleval,vin,event_time_stamp
0,45.0,omc854,2024-07-04 12:00:00.027
1,43.0,omc895,2024-07-04 00:00:05.667
2,55.0,omc561,2024-07-04 00:00:02.373
3,NaN,omc627,2024-07-04 00:00:00.707
4,0.0,omc536,2024-07-04 00:00:05.340
...,...,...,...
256792,73.0,omc118,2024-07-04 23:22:46.287
256793,77.0,omc152,2024-07-04 23:40:41.587
256794,52.0,omc552,2024-07-04 00:06:36.450
256795,81.0,omc152,2024-07-04 01:37:33.000


In [27]:
master_df


,vin,plant_id,site_name,total_cap_in_kwp,cap_in_kva,cap_in_kw,fuel_cap_in_litres,total_ah,battery_cap_in_kwh
0,omc895,224,Atia Majhawan,37.50,25.0,20.0,250.0,800.0,115.2
1,omc947,226,Bahuti,28.76,25.0,20.0,250.0,800.0,115.2
2,omc497,229,Bajitpur,31.18,25.0,20.0,250.0,800.0,115.2
3,omc773,164,Chhanaiya,22.68,30.0,24.0,250.0,800.0,115.2
4,omc501,227,Gursanda,34.02,25.0,20.0,250.0,800.0,115.2
...,...,...,...,...,...,...,...,...,...
381,omc872,2084,Dholai Kala,85.80,40.0,32.0,500.0,600.0,144.0
382,omc422,2091,Lalpurkarauta,46.20,40.0,32.0,500.0,600.0,144.0
383,omc787,2105,Daphar pur,50.05,50.0,40.0,500.0,600.0,144.0
384,omc942,2132,Saraurakala,50.05,40.0,32.0,500.0,600.0,144.0


In [28]:
# Filter the DataFrame based on the 'vin' value
fuel_cap_value = master_df.loc[master_df['vin'] == 'omc890', 'fuel_cap_in_litres']

# Print the value
print(fuel_cap_value)


331    250.0
Name: fuel_cap_in_litres, dtype: float64


In [ ]:


def find_consecutive_occurrences(data):
    if not data:
        return []
    result = []
    current_number = data[0]
    count = 1
    for num in data[1:]:
        if num == current_number:
            count += 1
        else:
            result.append((current_number, count))
            current_number = num
            count = 1
    result.append((current_number, count))
    
    return result



def fuel_tank(fuel_level_df):
    fuelleval_dict={}
    for vin in fuel_level_df['vin'].unique():
        fuel_composition=np.array([0],dtype='float64')
        fuel_filling=np.array([0],dtype='float64')
        tmp_df=fuel_level_df[fuel_level_df['vin']==vin].sort_values(by='event_time_stamp')
        d_fuel_level=tmp_df['fuelleval'].astype(float)
        fuel_level_list= d_fuel_level.to_list()
        fuel_level_list=find_consecutive_occurrences(fuel_level_list)
        fuel_level_list=[item[0] for item in fuel_level_list if item[1] >= 7]
        fuel_leve_df=pd.DataFrame(fuel_level_list)
        for df in fuel_leve_df.diff().values[1:]:
  
            if df <0:
                fuel_composition+= (df * -1)
            else:
                fuel_filling+=df

        
        fuelleval_dict[vin]={'fuel_composition':float(fuel_composition[0]),'fuel_filling':float(fuel_filling[0])}    
    df = pd.DataFrame.from_dict(fuelleval_dict, orient='index')
    df = df.reset_index().rename(columns={'index': 'vin'})
                
    return df       
datax=fuel_tank(fuel_level_df)